# Small Experiments Pertaining to Shaders for Scientific Visualisation
The contents of this notebook is neither documentation nor does it serve as permanent unit tests for the project. It's purely kept as a record of some of the experimentation done while working on this project.

Disclaimer of originality: Some of the code snippets in this notebook are derived from examples found online and are not original work.

### Custom HTML Output in Jupyter

In [2]:
class CustomHTMLObject:
    def __init__(self, name: str, colour: str):
        """
        
        :param name: The name of the object
        :param colour: The hex colour code of the object (eg: "#112233")
        """
        self.name = name
        self.colour = colour
        
    def _repr_html_(self) -> str:
        """
        Hooks into Jupyter notebook rich display system, which calls _repr_html_ by
        default if an object is returned at the end of a cell.
        """
        return f"""
        <div class="custom-html-object">
            <div style="color: {self.colour};font-size: large;">{self.name}</div>
        </div>"""
    
# Now return a CustomHTMLObject to the cell
CustomHTMLObject("Custom HTML is cool!", "#ff2010")

### Running OpenGL Headless

In [1]:
%pip install moderngl

     ---------------------------------------- 0.0/106.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/106.8 kB ? eta -:--:--
     -------------------------------------- 106.8/106.8 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for glcontext<3,>=2.3.6 from https://files.pythonhosted.org/packages/75/65/a4cd375ff65099f9b6203a960337c24d21d12162107b0fe4f55dd617d2e8/glcontext-2.4.0-cp310-cp310-win_amd64.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from PIL import Image
import moderngl

In [34]:
# This snippet for detecting the python environment is taken from: https://github.com/InsightSoftwareConsortium/itkwidgets/blob/main/itkwidgets/integrations/environment.py
# Licensed under the Apache License 2.0: https://github.com/InsightSoftwareConsortium/itkwidgets/blob/main/LICENSE
from enum import Enum
from importlib import import_module
import sys


class Env(Enum):
    JUPYTER_NOTEBOOK = 'notebook'
    JUPYTERLAB = 'lab'
    JUPYTERLITE = 'lite'
    SAGEMAKER = 'sagemaker'
    HYPHA = 'hypha'
    COLAB = 'colab'


def find_env():
    try:
        from google.colab import files
        return Env.COLAB
    except:
        try:
            from IPython import get_ipython
            parent_header = get_ipython().parent_header
            username = parent_header['header']['username']
            if username == '':
                return Env.JUPYTERLAB
            elif username == 'username':
                return Env.JUPYTER_NOTEBOOK
            else:
                return Env.SAGEMAKER
        except:
            import sys
            if sys.platform == 'emscripten':
                return Env.JUPYTERLITE
            return Env.HYPHA


ENVIRONMENT = find_env()

In [35]:
# Setup the OpenGL context
if ENVIRONMENT == Env.COLAB:
    # In Google Colab we need to explicitly specify the EGL backend, otherwise it tries (and fails) to use X11
    ctx = moderngl.create_context(standalone=True, backend="egl")
else:
    # Otherwise let ModernGL try to automatically determine the correct backend
    ctx = moderngl.create_context(standalone=True)
print(f"Got OpenGL context:\n\tGL_VENDOR={ctx.info['GL_VENDOR']}\n\tGL_RENDERER={ctx.info['GL_RENDERER']}\n\tGL_VERSION={ctx.info['GL_VERSION']}")
from pprint import pprint
pprint(ctx.info)

resolution = (min(1024, ctx.info["GL_MAX_VIEWPORT_DIMS"][0]), 
              min(1024, ctx.info["GL_MAX_VIEWPORT_DIMS"][1]))
fbo = ctx.simple_framebuffer(resolution, components=4)
fbo.use()

# Create a vertex buffer
vertices = np.array([
   # X      Y      R    G    B
    -1.0,  -1.0,   1.0, 0.0, 0.0,
     1.0,  -1.0,   0.0, 1.0, 0.0,
     0.0,   1.0,   0.0, 0.0, 1.0],
    dtype='f4',
)

prog = ctx.program(vertex_shader="""
#version 330
in vec2 in_vert;
in vec3 in_color;
out vec3 color;
out vec2 position;
void main() {
    gl_Position = vec4(in_vert, 0.0, 1.0);
    color = in_color;
    position = in_vert*0.5+0.5;
}
""",
    fragment_shader="""
#version 330
out vec4 fragColor;
in vec3 color;
in vec2 position;

uniform vec2 iResolution;

float amod(float x, float y)
{
    return x - y * floor(x/y);
}

vec4 mainImage(in vec2 fragCoord)
{
    // Normalized pixel coordinates (from 0 to 1)
    vec2 uv = fragCoord/iResolution.xy;

    float coord = floor(fragCoord.x) + floor(fragCoord.y/10.);
    //vec3 col = vec3(amod(coord, 16.)>=8.?1.:0., amod(coord, 32.)>=16.?1.:0., amod(coord, 64.)>=32.?1.:0.);
    //col = amod(coord, 128.)>64.?(col*0.3333+.3333):col;

    vec3 col = vec3(amod(coord, 64.) >= 56. ? 1. : 0.,
                    amod(coord + 16., 64.) >= 56. ? 1. : 0.,
                    amod(coord + 32., 64.) >= 56. ? 1. : 0.);
    col += amod(coord + 48., 64.) >= 56. ? 1. : 0.;
    col = amod(coord, 128.) > 64. ? (col * 0.3333 + .3333) : col;

    // Output to screen
    return vec4(col,1.0);
}

void main() {
    fragColor = mainImage(position * iResolution) + vec4(color, 1.0)*0.01;
    //fragColor = vec4(color, 1.0);
}
""",
)

# Set uniforms
prog["iResolution"].value = resolution

# Assign buffers and render
vao = ctx.simple_vertex_array(prog, ctx.buffer(vertices), 'in_vert', 'in_color')
vao.render(mode=moderngl.TRIANGLES)

# Save output to png
print("Saving image...")
image = Image.frombytes('RGBA', resolution, fbo.read(components=4))
image = image.transpose(Image.FLIP_TOP_BOTTOM)
image.save('openglHeadlessExample.png', format='png')
print(f"Saved to 'openglHeadlessExample.png'!")

Got OpenGL context:
	GL_VENDOR=NVIDIA Corporation
	GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
	GL_VERSION=3.3.0 NVIDIA 536.23
{'GL_ALIASED_LINE_WIDTH_RANGE': (1.0, 10.0),
 'GL_CONTEXT_PROFILE_MASK': 1,
 'GL_DOUBLEBUFFER': True,
 'GL_MAJOR_VERSION': 3,
 'GL_MAX_3D_TEXTURE_SIZE': 16384,
 'GL_MAX_ARRAY_TEXTURE_LAYERS': 2048,
 'GL_MAX_CLIP_DISTANCES': 8,
 'GL_MAX_COLOR_ATTACHMENTS': 8,
 'GL_MAX_COLOR_TEXTURE_SAMPLES': 32,
 'GL_MAX_COMBINED_FRAGMENT_UNIFORM_COMPONENTS': 233472,
 'GL_MAX_COMBINED_GEOMETRY_UNIFORM_COMPONENTS': 231424,
 'GL_MAX_COMBINED_TEXTURE_IMAGE_UNITS': 192,
 'GL_MAX_COMBINED_UNIFORM_BLOCKS': 84,
 'GL_MAX_COMBINED_VERTEX_UNIFORM_COMPONENTS': 233472,
 'GL_MAX_CUBE_MAP_TEXTURE_SIZE': 32768,
 'GL_MAX_DEPTH_TEXTURE_SAMPLES': 32,
 'GL_MAX_DRAW_BUFFERS': 8,
 'GL_MAX_DUAL_SOURCE_DRAW_BUFFERS': 1,
 'GL_MAX_ELEMENTS_INDICES': 1048576,
 'GL_MAX_ELEMENTS_VERTICES': 1048576,
 'GL_MAX_FRAGMENT_INPUT_COMPONENTS': 128,
 'GL_MAX_FRAGMENT_UNIFORM_BLOCKS': 14,
 'GL_MAX_FRAGMENT_UNIFO

In [37]:
print(ctx.extensions)
print(ctx.viewport)
print(ctx.extra)

{'GL_ARB_texture_non_power_of_two', 'GL_NV_timeline_semaphore', 'GL_SGIS_texture_lod', 'GL_EXT_gpu_program_parameters', 'GL_EXT_draw_instanced', 'GL_ARB_texture_env_dot3', 'GL_EXT_shader_image_load_formatted', 'GL_ARB_transpose_matrix', 'GL_EXT_memory_object', 'GL_NV_fragment_shader_interlock', 'GL_ARB_texture_filter_minmax', 'GL_ARB_internalformat_query2', 'GL_NV_transform_feedback2', 'GL_NV_memory_attachment', 'GL_ARB_robust_buffer_access_behavior', 'GL_ARB_polygon_offset_clamp', 'GL_NV_depth_buffer_float', 'GL_ARB_draw_buffers_blend', 'GL_NV_blend_square', 'GL_EXT_fog_coord', 'GL_EXT_pixel_buffer_object', 'GL_ARB_copy_image', 'GL_ARB_seamless_cubemap_per_texture', 'GL_NVX_linked_gpu_multicast', 'GL_ARB_depth_clamp', 'GL_NV_point_sprite', 'GL_NV_half_float', 'GL_ARB_timer_query', 'GL_ARB_post_depth_coverage', 'GL_ARB_shader_precision', 'GL_ARB_shader_viewport_layer_array', 'GL_ARB_occlusion_query', 'GL_NV_shader_atomic_fp16_vector', 'GL_EXT_blend_func_separate', 'GL_EXT_texture_shado

Exciting! So it seems that headless OpenGL rendering does indeed work with Google Colab (even when using a Colab session without a GPU surprisingly). Another thing to keep in mind though is that different platforms will have different capabilities, and we'll have to work out how to either handle those as errors (trying to allocate too many OpenGL buffers/samplers) or provide fallbacks (eg: tiled rendering for cases where the resolution is too high (on my machine the limit is 32k\*32k but on Colab the limit is 16k\*16k (for framebuffers and textures)). 

Don't know if this will present a problem, but it could be useful to hint to the graphics driver which GPU we want to use in multi GPU systems (such as laptops). As it turns out this is actually not standardised at all, and quite difficult in OpenGL since we don't create the context with a specific adapter (unlike DirectX). On Windows when using an Nvidia GPU we can set the "SHIM_MCCOMPAT=0x800000001" environment variable to specify the dedicated GPU; I'm not sure this works on other platforms, it's not well documented. It seems to me that on Google Colab at the moment, it only uses the software renderer with OpenGL (the Mesa llvmpipe driver) even when a GPU environment is used. For the purposes of development and testing this is probably _fine_, but to actually be useful we should really try to get GPU support working. From what I've read, it should just work ([https://stackoverflow.com/a/60937358](https://stackoverflow.com/a/60937358)) so I'm not sure what's wrong...

# ModernGL Threading

In [11]:
#  Copyright (c) 2023 Thomas Mathieson.
#  Distributed under the terms of the MIT license.
import logging
import time

import moderngl
import numpy as np


class SSVRenderOpenGL:
    """
    A rendering backend for SSV based on OpenGL
    """

    def __init__(self, resolution):
        self.resolution = resolution
        self.__create_context()
        self.start_time = time.time()

    def __create_context(self):
        """
        Creates an OpenGL context and a framebuffer.
        """
        self.ctx = moderngl.create_context(standalone=True)

        # To use moderngl with threading we need call the garbage collector manually
        self.ctx.gc_mode = "context_gc"

        resolution = (min(self.resolution[0], self.ctx.info["GL_MAX_VIEWPORT_DIMS"][0]),
                      min(self.resolution[1], self.ctx.info["GL_MAX_VIEWPORT_DIMS"][1]))
        self.fbo = self.ctx.simple_framebuffer(resolution, components=4)
        self.fbo.use()

    def log_context_info(self, full=False):
        """
        Logs the OpenGL information to the console for debugging.
        :param full: whether to log *all* of the OpenGL context information (including extensions)
        """
        print(f"Got OpenGL context:\n"
            f"\tGL_VENDOR={self.ctx.info['GL_VENDOR']}\n"
            f"\tGL_RENDERER={self.ctx.info['GL_RENDERER']}\n"
            f"\tGL_VERSION={self.ctx.info['GL_VERSION']}")
        if full:
            from pprint import pformat
            info = pformat(self.ctx.info, indent=4)
            print(f"Full info: \n{info}")
            extensions = pformat(self.ctx.extensions, indent=4)
            print(f"GL Extensions: \n{extensions}")

    def register_shader(self):
        ...

    def dbg_render_test(self):
        # Create a vertex buffer
        vertices = np.array([
            # X      Y      R    G    B
            -1.0, -1.0, 1.0, 0.0, 0.0,
            1.0, -1.0, 0.0, 1.0, 0.0,
            0.0, 1.0, 0.0, 0.0, 1.0],
            dtype='f4',
        )

        prog = self.ctx.program(vertex_shader="""
        #version 330
        in vec2 in_vert;
        in vec3 in_color;
        out vec3 color;
        out vec2 position;
        void main() {
            gl_Position = vec4(in_vert, 0.0, 1.0);
            color = in_color;
            position = in_vert*0.5+0.5;
        }
        """,
                                fragment_shader="""
        #version 330
        out vec4 fragColor;
        in vec3 color;
        in vec2 position;

        uniform vec2 iResolution;
        uniform float iTime;

        float amod(float x, float y)
        {
            return x - y * floor(x/y);
        }

        vec4 mainImage(in vec2 fragCoord)
        {
            // Normalized pixel coordinates (from 0 to 1)
            vec2 uv = fragCoord/iResolution.xy;

            float coord = floor(fragCoord.x) + floor(fragCoord.y/10.) + floor(iTime);
            //vec3 col = vec3(amod(coord, 16.)>=8.?1.:0., amod(coord, 32.)>=16.?1.:0., amod(coord, 64.)>=32.?1.:0.);
            //col = amod(coord, 128.)>64.?(col*0.3333+.3333):col;

            vec3 col = vec3(amod(coord, 64.) >= 56. ? 1. : 0.,
                            amod(coord + 16., 64.) >= 56. ? 1. : 0.,
                            amod(coord + 32., 64.) >= 56. ? 1. : 0.);
            col += amod(coord + 48., 64.) >= 56. ? 1. : 0.;
            col = amod(coord, 128.) > 64. ? (col * 0.3333 + .3333) : col;

            // Output to screen
            return vec4(col,1.0);
        }

        void main() {
            fragColor = mainImage(position * iResolution) + vec4(color, 1.0)*0.01;
            //fragColor = vec4(color, 1.0);
        }
        """)

        # Set uniforms
        prog["iResolution"].value = self.resolution
        prog["iTime"].value = time.time() - self.start_time

        # Assign buffers and render
        vao = self.ctx.simple_vertex_array(prog, self.ctx.buffer(vertices), 'in_vert', 'in_color')
        vao.render(mode=moderngl.TRIANGLES)

    def get_frame(self):
        """
        Gets the current contents of the frame buffer as a byte array.
        :return: the contents of the frame buffer as a bytearray in the ``RGBA`` format.
        """
        return self.fbo.read(components=4)


In [12]:
import threading


renderer = SSVRenderOpenGL((640,480))

def __render_loop(exit_flag):
    target_framerate = 60
    last_frame_time = time.time()
    timeout = 0
    delta_time = 1/target_framerate
    while not exit_flag.wait(timeout=timeout):
        renderer.dbg_render_test()

        current_time = time.time()
        delta_time = current_time - last_frame_time
        timeout = max(1/target_framerate - delta_time, 0)
        last_frame_time = current_time
        break

def run():
    exit_flag = threading.Event()
    thread = threading.Thread(target=__render_loop, args=(exit_flag,))

    # __render_loop(exit_flag)

    thread.start()
    # self.__render_loop(exit_flag)
    
run()

Traceback (most recent call last):
  File "C:\Users\Thoma\.conda\envs\IndProject\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Thoma\AppData\Local\Temp\ipykernel_49268\2383715671.py", line 12, in __render_loop
  File "C:\Users\Thoma\AppData\Local\Temp\ipykernel_49268\740362812.py", line 63, in dbg_render_test
  File "C:\Users\Thoma\.conda\envs\IndProject\lib\site-packages\moderngl\__init__.py", line 1939, in program
    res.mglo, res._members, res._subroutines, res._geom, res._glo = self.mglo.program(
_moderngl.Error: cannot create program


## Multiprocessing Experiments

In [12]:
from multiprocessing import Process, Queue

class FooMP:
    def __init__(self):
        print(self, f"FooMP init")
        proc1 = Process(target=self.test1)
        proc1.start()
        proc1.join()
    
    def test(self):
        print(self, "test")
        
    def test1(self):
        print(self, "test1")
        
foo = FooMP()

proc = Process(target=foo.test)
proc.start()
proc.join()
print("Test")

<__main__.FooMP object at 0x00000283BE086F50> FooMP init
Test


## Shader Preprocessing Experiments

In [13]:
%pip install pcpp

     ---------------------------------------- 0.0/91.1 kB ? eta -:--:--
     ------------ ------------------------- 30.7/91.1 kB 445.2 kB/s eta 0:00:01
     ---------------------------------------- 91.1/91.1 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [77]:
import pcpp
import argparse

class ShaderPreprocessor(pcpp.Preprocessor):
    def __init__(self):
        super(ShaderPreprocessor, self).__init__()
        # Define any built in macros/includes
        
        # Pragma parser
        self.pragma_parse = argparse.ArgumentParser(prog="SSV Shader Preprocessor")
        sub_parsers = self.pragma_parse.add_subparsers(help="template generation command help", dest="command")
        pixel_parser = sub_parsers.add_parser("pixel", help="define the entrypoint for a pixel shader")
        pixel_parser.add_argument("function", type=str)
        vertex_parser = sub_parsers.add_parser("vertex", help="define the entrypoint for a vertex shader")
        vertex_parser.add_argument("function", type=str)
        sdf_parser = sub_parsers.add_parser("sdf", help="define the entrypoint for an sdf shader")
        sdf_parser.add_argument("function", type=str)
        sdf_parser.add_argument("--sdf_normal", type=str)
        sdf_parser.add_argument("--display_mode", choices=["solid", "xray", "isolines", "2d"])
        sdf_parser.add_argument("--surface_offset", type=str)
        self.pragma_args = []
        
    def on_directive_unknown(self, directive, toks, ifpassthru, precedingtoks):
        if directive.value != "pragma":
            return super(ShaderPreprocessor, self).on_directive_unknown(directive, toks, ifpassthru, precedingtoks)
        
        if not (len(toks) > 2 and toks[0].value == "SSV"):
            return super(ShaderPreprocessor, self).on_directive_unknown(directive, toks, ifpassthru, precedingtoks)
        
        tok_strs = [t.value for t in toks[2:]]
        # print(f"Found SSV pragma: {''.join(tok_strs)}")
        self.pragma_args.append(''.join(tok_strs))
        
    def write(self, oh=sys.stdout):
        # Prefixed template code
        
        # Output the preprocessed code
        super(ShaderPreprocessor, self).write(oh)
        # Suffixed template code
        
        print("\n############\n")
        print("Found following SSV pragmas: ")
        for pragma in self.pragma_args:
            args = self.pragma_parse.parse_args(pragma.split())
            print("\t", args)

In [78]:
shader = """
#pragma SSV pixel main_ps
#pragma SSV vertex main_vs
#pragma SSV sdf map --display_mode solid --surface_offset u_offset --sdf_normal map_nrm
#define saturate(x) clamp((x), 0., 1.)

void main_vs(in vec4 pos, out VertexOut out) {
    out.pos = pos * mat_mvp;
    out.color = pos.xy;
}

void main_ps(in VertexOut vert, in vec2 fragCoord, out vec4 fragColor) {
    fragColor = saturate(vert.color);
}

float map(vec3 pos) {
    pos = fract(pos) * 2. - 1.;
    return length(pos) - 0.25;
}

vec3 map_nrm(vec3 pos) {
    return vec3(1., 0., 0.);
}
"""

In [79]:
preproc = ShaderPreprocessor()
preproc.parse(shader)
preproc.write(sys.stdout)


#pragma SSV pixel main_ps
#pragma SSV vertex main_vs
#pragma SSV sdf map --display_mode solid --surface_offset u_offset --sdf_normal map_nrm


void main_vs(in vec4 pos, out VertexOut out) {
    out.pos = pos * mat_mvp;
    out.color = pos.xy;
}

void main_ps(in VertexOut vert, in vec2 fragCoord, out vec4 fragColor) {
    fragColor = clamp((vert.color), 0., 1.);
}

float map(vec3 pos) {
    pos = fract(pos) * 2. - 1.;
    return length(pos) - 0.25;
}

vec3 map_nrm(vec3 pos) {
    return vec3(1., 0., 0.);
}

############

Found following SSV pragmas: 
	 Namespace(command='pixel', function='main_ps')
	 Namespace(command='vertex', function='main_vs')
	 Namespace(command='sdf', function='map', sdf_normal='map_nrm', display_mode='solid', surface_offset='u_offset')


In [13]:
import argparse

template_args_src = "pixel -a entrypoint -a _varying_struct --type str"

parser = argparse.ArgumentParser("SSV Template Engine")
parser.add_argument("template_name", type=str)
parser.add_argument("--argument", "-a", nargs="+", action="append")

param_parser = argparse.ArgumentParser("SSV Template Engine Argument")
param_parser.add_argument("name", type=str)
param_parser.add_argument("--type", "-t", type=str)

parser.parse_args(template_args_src.split())

Namespace(template_name='pixel', argument=[['entrypoint'], ['_varying_struct', 'type', 'str']])

## HTTP Server Experiments

In [31]:
%pip install av


   ---------------------------------------- 0.0/25.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.9 MB 1.4 MB/s eta 0:00:19
    --------------------------------------- 0.3/25.9 MB 5.1 MB/s eta 0:00:05
   - -------------------------------------- 0.8/25.9 MB 6.6 MB/s eta 0:00:04
   -- ------------------------------------- 1.4/25.9 MB 8.1 MB/s eta 0:00:04
   --- ------------------------------------ 2.1/25.9 MB 9.5 MB/s eta 0:00:03
   ---- ----------------------------------- 2.9/25.9 MB 11.0 MB/s eta 0:00:03
   ------ --------------------------------- 3.9/25.9 MB 12.5 MB/s eta 0:00:02
   ------- -------------------------------- 5.0/25.9 MB 13.9 MB/s eta 0:00:02
   --------- ------------------------------ 6.4/25.9 MB 15.7 MB/s eta 0:00:02
   ------------ --------------------------- 8.1/25.9 MB 17.8 MB/s eta 0:00:02
   --------------- ------------------------ 10.0/25.9 MB 19.9 MB/s eta 0:00:01
   ------------------ --------------------- 12.2/25.9 MB 29.8 MB/s eta 0:00

In [1]:
%pip install portpicker

Note: you may need to restart the kernel to use updated packages.


In [31]:
import numpy as np
import av
import io
from typing import Optional, AnyStr
import time


class HTTPStreamedIO(io.RawIOBase):
    name: str = "stream.ts"
    written: int = 0
    
    def writable(self) -> bool:
        return True
    
    def readable(self) -> bool:
        return False
    
    def write(self, __b: AnyStr) -> Optional[int]:
        # print(f"  Writing: {len(__b)} bytes...")
        self.written += len(__b)
        return len(__b)


# Code modified from: https://pyav.org/docs/stable/cookbook/numpy.html#generating-video
duration = 2
fps = 60
total_frames = int(duration * fps)

io_stream = HTTPStreamedIO()
# container = av.open(io_stream, format="null", mode="w")
# container = av.open(io_stream, format="data", mode="w")
# container = av.open(io_stream, mode="w")
container = av.open("test3.mkv", mode="w")
container.flags |= container.flags.FLUSH_PACKETS
container.flags |= container.flags.NOBUFFER
container.flags |= container.flags.CUSTOM_IO

stream = container.add_stream("hevc", rate=fps)
# stream.format = av.format.ContainerFormat("data", "w")
stream.width = 480
stream.height = 320
stream.pix_fmt = "yuv420p"
# stream.pix_fmt = "yuv444p"
# stream.codec_context.flags |= stream.codec_context.flags.LOW_DELAY
stream.options = {"g": "1", "zerolatency": "1", "lag-in-frames": "1"}
# stream.options["color_range"] = "2"
# stream.options["strict_std_compliance"] = "unofficial"

imgs = []
for frame_i in range(total_frames):
    img = np.empty((480, 320, 3))
    img[:, :, 0] = 0.5 + 0.5 * np.sin(2 * np.pi * (0 / 3 + frame_i / total_frames))
    img[:, :, 1] = 0.5 + 0.5 * np.sin(2 * np.pi * (1 / 3 + frame_i / total_frames))
    img[:, :, 2] = 0.5 + 0.5 * np.sin(2 * np.pi * (2 / 3 + frame_i / total_frames))

    img = np.round(255 * img).astype(np.uint8)
    img = np.clip(img, 0, 255)
    imgs.append(img)

start_time = time.perf_counter()
for frame_i in range(total_frames):
    img = imgs[frame_i]
    frame = av.VideoFrame.from_ndarray(img, format="rgb24")
    for i, packet in enumerate(stream.encode(frame)):
        print(f"Encoded frame {frame_i} packet {i} ({len(bytes(packet))})...")
        container.mux(packet)

# Flush stream
for i, packet in enumerate(stream.encode()):
    # print(f"Flushing packet {i}...")
    container.mux(packet)

# Close the file
container.close()
encode_time = time.perf_counter() - start_time
print(f"Wrote {total_frames} frames into {io_stream.written} bytes in {encode_time*1000:.2f} ms. {encode_time*1000/total_frames:.2f}/{1000/fps:.2f} ms/frame")

Encoded frame 4 packet 0 (2483)...
Encoded frame 5 packet 0 (2474)...
Encoded frame 6 packet 0 (2474)...
Encoded frame 7 packet 0 (2474)...
Encoded frame 8 packet 0 (2480)...
Encoded frame 9 packet 0 (2530)...
Encoded frame 10 packet 0 (2525)...
Encoded frame 11 packet 0 (2553)...
Encoded frame 12 packet 0 (2540)...
Encoded frame 13 packet 0 (2542)...
Encoded frame 14 packet 0 (2549)...
Encoded frame 15 packet 0 (2546)...
Encoded frame 16 packet 0 (2543)...
Encoded frame 17 packet 0 (2544)...
Encoded frame 18 packet 0 (2544)...
Encoded frame 19 packet 0 (2541)...
Encoded frame 20 packet 0 (2542)...
Encoded frame 21 packet 0 (2542)...
Encoded frame 22 packet 0 (2717)...
Encoded frame 23 packet 0 (2547)...
Encoded frame 24 packet 0 (2543)...
Encoded frame 25 packet 0 (2543)...
Encoded frame 26 packet 0 (2717)...
Encoded frame 27 packet 0 (2542)...
Encoded frame 28 packet 0 (2542)...
Encoded frame 29 packet 0 (2541)...
Encoded frame 30 packet 0 (2546)...
Encoded frame 31 packet 0 (2542)..

In [34]:
from PIL import Image
from io import BytesIO

from http.server import HTTPServer, BaseHTTPRequestHandler
import portpicker
import threading
import socketserver
import socket

img = Image.open("openglHeadlessExample.png")
img = img.convert('RGB')
img_bytes = BytesIO()
quality = 75
img.save(img_bytes, format='jpeg', quality=quality)
img_bytes = img_bytes.getvalue()
print(len(img_bytes))

class SSVHTTPHandler(BaseHTTPRequestHandler):
  def do_GET(self):
      self.send_response(200)
      self.send_header('x-colab-notebook-cache-control', 'no-cache')
      self.send_header("content-type", "image/jpg")
      # self.send_header("content-length", str(len(img_bytes)))
      self.end_headers()
      self.wfile.write(img_bytes)

port = portpicker.pick_unused_port()
print(f"Starting server on: localhost:{port}")

def http_server_run():
    http_server = HTTPServer(('localhost', port), SSVHTTPHandler)
    http_server.serve_forever()

# http_server_run()
thread = threading.Thread(target=http_server_run)
thread.start()

151907
Starting server on: localhost:49460


In [70]:
from IPython.display import display, Javascript, HTML

display(HTML("""
<div id='ssv-test'>Hello</div>
<img id='ssv-test1' src='https://localhost:58583/'></img>
"""))

display(Javascript('''
let e = document.getElementById("ssv-test");
e.style.fontSize = "20pt";
'''))

<IPython.core.display.Javascript object>

In [71]:
import asyncio
from websockets.sync.server import serve
import portpicker
import threading

def echo(websocket):
    for message in websocket:
        websocket.send(message)

port = portpicker.pick_unused_port()
print(f"Starting server on: localhost:{port}")

def main():
    with serve(echo, "localhost", port) as server:
        server.serve_forever()

# asyncio.run(main())
thread = threading.Thread(target=main)
thread.start()

Starting server on: localhost:53378


In [69]:
from IPython.display import display, Javascript, HTML

display(HTML("""
<div id='ssv-test'>Hello</div>
"""))
display(Javascript(f"""
const elm = document.getElementById("ssv-test").innerText = "Connecting to sever...";

// Create WebSocket connection.
const socket = new WebSocket("ws://127.0.0.1:{port}");

// Connection opened
socket.addEventListener("open", (event) => {{
  socket.send("Hello Server!");
  const elm = document.getElementById("ssv-test").innerText = "Connected to sever!";
}});

socket.addEventListener("message", (event) => {{
  console.log("Message from server ", event.data);
  const elm = document.getElementById("ssv-test").innerText = event.data;
}});
"""))

<IPython.core.display.Javascript object>

In [1]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import portpicker
import threading
import time

class SSVHTTPHandler1(BaseHTTPRequestHandler):
  def do_GET(self):
      self.send_response(200)
      self.send_header('x-colab-notebook-cache-control', 'no-cache')
      self.send_header("content-type", "text/plain")
      self.send_header("Access-Control-Allow-Origin", "*")
      # self.send_header("content-length", str(12))
      self.end_headers()
      print("Starting sending...")
      # time.sleep(1)
      self.wfile.write(b"Testing ")
      self.wfile.flush()
      time.sleep(1)
      self.wfile.write(b"1")
      time.sleep(1)
      self.wfile.write(b"2")
      time.sleep(1)
      self.wfile.write(b"3")
      time.sleep(1)
      self.wfile.write(b"4")
      print("Finished sending...")

port = portpicker.pick_unused_port()
print(f"Starting server on: localhost:{port}")

if "http_server" in globals() and http_server is not None:
    http_server.shutdown()
http_server = None

def http_server_run():
    global http_server
    http_server = HTTPServer(('localhost', port), SSVHTTPHandler1)
    http_server.serve_forever()

# http_server_run()
thread = threading.Thread(target=http_server_run)
thread.start()

Starting server on: localhost:60793
